In [1]:
import tensorflow
tf = tensorflow
print(tensorflow.__version__)

2.1.0


In [2]:
with open('LOTR_Fellowship_Text.txt', 'rb') as fin:
    lotr_1_rtext_b = fin.read()
lotr_1_rtext = lotr_1_rtext_b.decode()
lotr_1_rtext[:1000]

'\r\nBook I\r\n\r\n \r\nChapter 1\r\nA Long-expected Party\r\n\r\nWhen Mr. Bilbo Baggins of Bag End announced that he would shortly be celebrating his eleventy-first birthday with a party of special magnificence, there was much talk and excitement in Hobbiton.\r\n\r\nBilbo was very rich and very peculiar, and had been the wonder of the Shire for sixty years, ever since his remarkable disappearance and unexpected return. The riches he had brought back from his travels had now become a local legend, and it was popularly believed, whatever the old folk might say, that the Hill at Bag End was full of tunnels stuffed with treasure. And if that was not enough for fame, there was also his prolonged vigour to marvel at. Time wore on, but it seemed to have little effect on Mr. Baggins. At ninety he was much the same as at fifty. At ninety-nine they began to call him well-preserved, but unchanged would have been nearer the mark. There were some that shook their heads and thought this was too muc

In [3]:
lotr_1_rtext.find('\xa0')

-1

In [ ]:
import re

In [4]:
lotr_1_books = re.split('\r\nBook [IVX]+\r\n',lotr_1_rtext)[1:]
len(lotr_1_books)
[x[:100] for x in lotr_1_books]

NameError: name 're' is not defined

In [ ]:
lotr_1_book_chapters = [re.split('Chapter \d+\r\n.*\r\n', x.strip())[1:] for x in lotr_1_books]
[[ch[:100]+'...'+ch[-100:] for ch in bk[:2]+bk[-2:]] for bk in lotr_1_book_chapters]

In [ ]:
lotr_1_book_ch_paras = [[re.split('\r\n\r\n\s*', ch.strip()) for ch in bk] for bk in lotr_1_book_chapters]
lotr_1_book_ch_paras[0][0][:4]

In [ ]:
pilcrow = '¶'
section = '§'
hyphen = '-' 
ndash = '\u2013' #'–'
mdash = '—'
elipses = '…'
print(hyphen, ndash, mdash)
hyphen == ndash

In [ ]:
# May be good to concatenate the newlines,
# which seem to mess up the later tokenisation, as well as a few other
# things: ". . ." as elipses (utf8 …), and "--" as mdash (—)
import re
def sentence_prep(sentence, debug=False):
    sentence = ' '.join(re.split("\s+", sentence))
    sentence = re.sub('\s*\. \. \.', ' … ',sentence)
    sentence = re.sub('----', ' — ',sentence)
    sentence = re.sub('---', ' — ',sentence)
    sentence = re.sub('--', ' — ',sentence)
    sentence = re.sub('“', ' “ ',sentence)
    sentence = re.sub('”', ' ” ',sentence)
    sentence = re.sub('‘', ' ‘ ',sentence)
    sentence = re.sub('’([a-z])', '\'\\1', sentence) #eg "it isn’t natural"/"you’ve" -> "isn't"/"you've"
    sentence = re.sub('’', ' ’ ',sentence)
    return sentence

print(sentence_prep(
    'This “should” get replaced . . . and even this--one',
    debug=True,
))

In [ ]:
#Split into sentences now
# Note full stop chars may not give the full sentences, eg
# "Mr. Bilbo Baggins"

#Import a premade sentence tokeniser
import nltk.data

#May need to download the data
nltk.download('punkt')
english_sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [9]:
from nltk.tokenize.punkt import PunktSentenceTokenizer
punkt_tokenizer = PunktSentenceTokenizer(lotr_1_rtext, verbose=1)
punkt_tokenizer


['\r\nBook I\r\n\r\n \r\nChapter 1\r\nA Long-expected Party\r\n\r\nWhen Mr. Bilbo Baggins of Bag End announced that he would shortly be celebrating his eleventy-first birthday with a party of special magnificence, there was much talk and excitement in Hobbiton.',
 'Bilbo was very rich and very peculiar, and had been the wonder of the Shire for sixty years, ever since his remarkable disappearance and unexpected return.',
 'The riches he had brought back from his travels had now become a local legend, and it was popularly believed, whatever the old folk might say, that the Hill at Bag End was full of tunnels stuffed with treasure.',
 'And if that was not enough for fame, there was also his prolonged vigour to marvel at.',
 'Time wore on, but it seemed to have little effect on Mr. Baggins.',
 'At ninety he was much the same as at fifty.',
 'At ninety-nine they began to call him well-preserved, but unchanged would have been nearer the mark.',
 'There were some that shook their heads and th

In [ ]:
lotr_prep_bk_ch_para_sens = [[
    [
        [sentence_prep(sen, debug=False) for sen in english_sentence_tokenizer.tokenize(para)]
        for para in ch
    ] for ch in lotr_book
] for lotr_book in lotr_1_book_ch_paras]
#aoav_prep_paras = [
#    sentence_prep(para, debug=False)
#    for ch in aoav_ch_paras
#    for para in ch
#]
lotr_prep_bk_ch_para_sens[0][0][:4]

In [ ]:
print( '\n-----\n'.join(lotr_prep_bk_ch_para_sens[0][0][1][:]) )
print('===')
print( '\n-----\n'.join(lotr_prep_bk_ch_para_sens[1][6][0][:]) ) 

In [ ]:
lotr_all_stringable_sentences = []
for book in lotr_prep_bk_ch_para_sens:
    for ch in book:
        sen_list = []
        for para in ch:
            for sen in para:
                sen_list.append(sen)
        lotr_all_stringable_sentences.append(sen_list)
lotr_all_stringable_sentences[1]

In [ ]:
#Start using tf for tokenization etc as well.
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(oov_token='<UNKNOWN>')

In [ ]:
lotr_all_sentences_strung = []
for sentence_list in lotr_all_stringable_sentences:
    lotr_all_sentences_strung.extend(sentence_list)

lotr_all_sentences_strung[:10]

In [ ]:
print("%d total sentences" % len(lotr_all_sentences_strung))
tokenizer.fit_on_texts(lotr_all_sentences_strung)
total_words = len(tokenizer.word_index) + 1 #word_index is a dict
print("%d total words" % total_words)

In [ ]:
#Inspect how this processes:
test_sentence = lotr_all_sentences_strung[2] + ' - then an OOV word and mdash: ' + mdash
print(test_sentence)
tseq = tokenizer.texts_to_sequences([test_sentence])[0]
print(tseq)
print(tokenizer.sequences_to_texts([tseq]))

In [ ]:
# v2 - try to make sentences by adding the full stop punctuation
DOT_STR = "·" #One that wont get filtered by the default tokenisation
tokenizer.fit_on_texts([DOT_STR])

DOT_TOKEN = tokenizer.texts_to_sequences([DOT_STR])[0][0]
print(DOT_STR, "->", DOT_TOKEN)

total_words = len(tokenizer.word_index) + 1 #word_index is a dict
print("%d total words" % total_words)


In [ ]:
#ngram - note that paragraphs are stringable, but probably best to exclude chapters
## modified From the colab:
# create input sequences using list of tokens

DOT_SENTENCES = True #Whether to include a dot token between sentences

token_strings_list = []
for stringable_sentences in lotr_all_stringable_sentences:
    tokenized_sentences = tokenizer.texts_to_sequences(stringable_sentences)
    #print(stringable_sentences, tokenized_sentences)
    #break
    tseq = []
    for tlist in tokenized_sentences:
        tseq.extend(tlist)
        if DOT_SENTENCES:
            tseq.append(DOT_TOKEN)
    token_strings_list.append(tseq)

include_sentences_alone = True
if include_sentences_alone:
    for sentence_list in lotr_all_stringable_sentences:
        tokenized_sentences = tokenizer.texts_to_sequences(sentence_list)
        token_strings_list.extend(tokenized_sentences)


In [ ]:
longest_sentence_len = max([(len(x),x) for x in token_strings_list])
print(
    len(token_strings_list), "sentence lists", ",",
    longest_sentence_len[0], "is longest len", " --> ",
    longest_sentence_len[1][:30],
    tokenizer.sequences_to_texts([longest_sentence_len[1][:30]])[0]
)


In [ ]:
#Ought to limit the back check, maybe to 30 words
MAX_ALLOWED_SEQ_LENGTH = 30 #then there is 1 for the output

In [ ]:
input_sequences=[]
for tseq in token_strings_list:
    for i in range(len(tseq)):
        #For each, collect the ngrams
        #And collect the [0:n] token 'n-grams'
        #  (feed in partial sentences as well)
        n_gram_sequence = tseq[max(0,i-MAX_ALLOWED_SEQ_LENGTH):i+1]
        input_sequences.append(n_gram_sequence)
    #print(tseq, n_gram_sequence, len(n_gram_sequence))
    #break

input_sequences[:4]

In [ ]:
#print(input_sentences[:2])
print(input_sequences[:20])
print(max(len(x) for x in input_sequences))

In [ ]:
#Prepare the sequences by padding them (left side zeros / "pre-padding")
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences



input_sequences_padded = np.array(
    pad_sequences(
        input_sequences,
        maxlen=MAX_ALLOWED_SEQ_LENGTH,
        padding='pre',
    )
)
input_sequences_padded

In [ ]:
input_sequences_padded.shape

In [ ]:
#This is enough for the prep.
# Now lets save, and then the data can be input into the processing script
#Need to save the tokeniser as well for encoding:
# then can use eg: tokenizer.texts_to_sequences([the_sentence])[0]

import pickle

with open('lotr_data_prep.pickle', 'wb') as handle:
    pickle.dump({
            'tokenizer': tokenizer,
            'train_tokens': input_sequences_padded,
        },
        handle,
    )